In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch_transformers

In [ ]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

In [ ]:
directory_path = '/content/drive/My Drive/Colab Notebooks/Writing-with-BERT-master'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movie_df = pd.read_csv(os.path.join(directory_path,'formatted_movie_lines_train.txt'), delimiter = '\n', error_bad_lines=False)
movie_lines_df = pd.DataFrame()

In [ ]:
movie_lines_df['CONVO'] = movie_df.iloc[:,0]

for i in range(len(movie_lines_df.iloc[:,0])):
  movie_lines_df.iloc[i,0] = movie_lines_df.iloc[i,0].replace('\t', ' ')

movie_lines_df['CONVO'] = movie_lines_df.CONVO.str.lower()
movie_lines_df['CONVO'] = movie_lines_df['CONVO'].map(lambda k: '[LINE] ' + str(k))
changed_text=movie_lines_df.CONVO.apply(lambda x:x+"\n"+"\n")


In [ ]:
movie_lines_df.head()

,CONVO
0,[LINE] you're asking me out. that's so cute. ...
1,"[LINE] no, no, it's my fault -- we didn't have..."
2,[LINE] why? unsolved mystery. she used to be ...
3,"[LINE] gosh, if only we could find kat a boyfr..."
4,[LINE] c'esc ma tete. this is my head right. ...


In [ ]:
open(os.path.join(directory_path,'data_lm_movie.txt'), "w").write(''.join(changed_text))

In [ ]:
#%cd /content/drive/My Drive/Bert Training/pytorch-transformers/examples/lm_finetuning
import os
os.listdir(os.getcwd())

['.config', 'drive', 'sample_data']

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Writing-with-BERT-master")

In [ ]:
#pregenerating training data

!python3 pregenerate_training_data.py --train_corpus data_lm_movie.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 5 --max_seq_len 128

In [ ]:
os.listdir(os.getcwd())

['.config', 'sample_data']

In [ ]:
#fine-tuning bert

!python3 finetune_on_pregenerated.py --pregenerated_data training/ --bert_model bert-base-uncased --do_lower_case --train_batch_size 16  --output_dir finetuned_lm/ --epochs 5

2021-03-25 16:34:42,192: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2021-03-25 16:34:42,197: Output directory (finetuned_lm) already exists and is not empty!
2021-03-25 16:34:42,470: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2021-03-25 16:34:42,774: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmprw2oqai9
100% 433/433 [00:00<00:00, 381620.85B/s]
2021-03-25 16:34:43,059: copying /tmp/tmprw2oqai9 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2021-03-25 16:34:43,059: creating metadata fi

In [ ]:
import torch
import random
import numpy as np
from pytorch_transformers import BertTokenizer, BertForMaskedLM
import nltk

In [ ]:
base_dir = '/content/drive/My Drive/Colab Notebooks/Writing-with-BERT-master/finetuned_lm'
tokenizer = BertTokenizer.from_pretrained(base_dir)
model = BertForMaskedLM.from_pretrained(base_dir, output_attentions=False)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def duplicates(lst, item):
	return [i for i, x in enumerate(lst) if x == item]

In [ ]:
def remove_duplicates(Text : str) -> str:
    '''
    This function simply removes unwanted duplicates from sentences
    '''

    sentence_list = sent_tokenize(Text)
    word_lists = []
    for sentence in sentence_list:
        for word in word_tokenize(sentence):
            word_lists.append(word)

    done = False

    while done == False:
      change = False
      for i in range(len(word_lists) - 1):
        if word_lists[i] == word_lists[i + 1]:
          word_lists.pop(i)
          change = True
          break

      if change == False:
        done = True

    string = ''
    for i in range(len(word_lists)):
      string += word_lists[i] + ' '

    return string

In [ ]:
def predict(starter, version):
  sentence = ""
  sentence_length = random.choice([6,7,8,9,10,11,12])
  
  filler = ' '.join(['MASK' for _ in range(int(sentence_length))])
  decoding_type = version # = 'random'
  sentence = "[CLS] " + starter + " " + filler + " . [SEP]"

  tokenized_text = tokenizer.tokenize(sentence)
  idxs = duplicates(tokenized_text, 'mask')
  for masked_index in idxs:
    tokenized_text[masked_index] = "[MASK]"	

  ##### LOOP TO CREATE TEXT #####
  generated = 0
  full_sentence = []
  while generated<int(sentence_length):
    mask_idxs = duplicates(tokenized_text, "[MASK]")
    
    if decoding_type =='left to right':
      focus_mask_idx = min(mask_idxs)
    else:
      focus_mask_idx = random.choice(mask_idxs)
    
    mask_idxs.pop(mask_idxs.index(focus_mask_idx))
    temp_tokenized_text = tokenized_text.copy()
    temp_tokenized_text = [j for i, j in enumerate(temp_tokenized_text) if i not in mask_idxs]
    temp_indexed_tokens = tokenizer.convert_tokens_to_ids(temp_tokenized_text)
    ff = [idx for idx, i in enumerate(temp_indexed_tokens) if i==103]
    temp_segments_ids = [0]*len(temp_tokenized_text)
    tokens_tensor = torch.tensor([temp_indexed_tokens])
    segments_tensors = torch.tensor([temp_segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    #TOP - k Sampling
    k=5
    predicted_index = random.choice(predictions[0, ff].argsort()[0][-k:]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    tokenized_text[focus_mask_idx] = predicted_token
    generated += 1
    text_output = ' '.join(tokenized_text[1:-1]).replace('[ review ]','')
    text_output = text_output.replace(' ##','')
    text_output = text_output.replace('fucking', 'f***ing')

    text_output = remove_duplicates(text_output)
  return text_output

In [ ]:
output = predict('The', 'random')
print(output)

the f***ing goddamn f***ing door . 


In [ ]:
import random
def generate_script(characters : list, number_of_responses : int, full_feeder : bool, version : str):

  '''
  This function uses BERT to generate a script based on a list of characters

  characters : list of character names of length 2
  number_of_responeses : length of script
  full_feeder : whether or not to use a predetermined first line
  version : version of BERT-decoder to use ('left to right' or 'random')
  '''

  #feeder_sentence = f'Can you believe it {characters[1]}?'
  feeder_sentences = [f'What is going on with you today {characters[1]}?',
                      'What is that over there?',
                      'How are you doing today?',
                      'Are you hungry?']



  helper_starter_list = ['we should',
                        'if we',
                        'can i',
                        'i cant',
                        'you cant',
                        'dont you',
                        'what was',
                        'next time',
                        'can you just',
                        'tell me',
                        'I can',
                        'well then',
                        'after that',
                        'however',
                        'there was',
                        'please']

  buffer = 0
  if full_feeder == True:
    buffer = 1
    feeder_sentence = random.choice(feeder_sentences)
  else:
    feeder_sentence = predict(random.choice(helper_starter_list), version)

  conversation = [feeder_sentence]
  previous_response = feeder_sentence

  i = 0
  for i in range(number_of_responses):
    if i % 2 == 0:
      next_response = predict(previous_response + random.choice(helper_starter_list), version)[len(previous_response):]
    else:
      next_response = predict(previous_response + random.choice(helper_starter_list), version)[len(previous_response):]
    i += 1

    conversation.append(next_response)
    previous_response = next_response

  i = 0

  for sentence in conversation:
    if i == 1:
      print(f'{characters[1]} : ' + sentence[buffer:])
      print()
    elif i % 2 == 0:
      print(f'{characters[0]} : ' + sentence)
      print()
    else:
      print(f'{characters[1]} : ' + sentence)
      print() 
    i += 1



In [ ]:
generate_script(characters = ['Shrek', 'Donkey'],
                number_of_responses = 2,
                full_feeder = True,
                version = 'left to right')

Shrek : Are you hungry?

Donkey :  tell me something important . okay . okay . 

Shrek : please listen closely now listen closely then go home . ok okay . 



In [ ]:
from nltk.translate import bleu_score as bleu

def self_bleu(sents):
    return bleu.corpus_bleu([[s for (j, s) in enumerate(sents) if j != i] for i in range(len(sents))], sents)


In [ ]:

highest_blue_score = 0
current_best_output = ''
for i in range(50):
  output = predict('Once upon a time', 'random')
  score = self_bleu(output)
  #print('Text : ',output)
  #print()
 # print(score)
#  print('------')
  if score > highest_blue_score:
    highest_blue_score = score
    current_best_output = output
  if score > 0.99:
    break

print('Best BLEU score : ', highest_blue_score)
print('Text : ', current_best_output)

Best BLEU score :  0.988021791350754
Text :  once upon a time she had probably just was staying somewhere down here and tonight . 


In [ ]:
lowest_blue_score = 1
current_worst_output = ''
for i in range(50):
  output = predict(' ', 'random')
  score = self_bleu(output)
  #print('Text : ',output)
  #print()
 # print(score)
#  print('------')
  if score < lowest_blue_score:
    lowest_blue_score = score
    current_worst_output = output
  if score < 0.01:
    break

print('Worst BLEU score : ', lowest_blue_score)
print('Text : ', current_worst_output)

Worst BLEU score :  0.7598356856515925
Text :  [ line p04 ? . 


In [ ]:
def generate_script_max(characters : list, number_of_responses : int, full_feeder : bool, version : str):

  '''
  This function uses BERT to generate a script based on a list of characters

  characters : list of character names of length 2
  number_of_responeses : length of script
  full_feeder : whether or not to use a predetermined first line
  version : version of BERT-decoder to use ('left to right' or 'random')
  '''

  #feeder_sentence = f'Can you believe it {characters[1]}?'
  feeder_sentences = [f'What is going on with you today {characters[1]}?',
                      'What is that over there?',
                      'How are you doing today?',
                      'Are you hungry?']



  helper_starter_list = ['we should',
                        'if we',
                        'can i',
                        'i cant',
                        'you cant',
                        'dont you',
                        'what was',
                        'next time',
                        'can you just',
                        'tell me',
                        'I can',
                        'well then',
                        'after that',
                        'however',
                        'there was',
                        'please']

  buffer = 0
  if full_feeder == True:
    buffer = 1
    feeder_sentence = random.choice(feeder_sentences)
  else:
    feeder_sentence = predict(random.choice(helper_starter_list), version)

  conversation = [feeder_sentence]
  previous_response = feeder_sentence

  i = 0
  for i in range(number_of_responses):
    if i % 2 == 0:

      highest_blue_score = 0
      current_best_next_response = ''
      for i in range(10):
        next_response = predict(previous_response + random.choice(helper_starter_list), version)[len(previous_response):]
        score = self_bleu(next_response)
        if score > highest_blue_score:
          highest_blue_score = score
          current_best_next_response = next_response
        if score > 0.99:
          break
    else:
      highest_blue_score = 0
      current_best_next_response = ''
      for i in range(10):
        next_response = predict(previous_response + random.choice(helper_starter_list), version)[len(previous_response):]
        score = self_bleu(next_response)
        if score > highest_blue_score:
          highest_blue_score = score
          current_best_next_response = next_response
        if score > 0.99:
          break
    i += 1

    conversation.append(current_best_next_response)
    previous_response = current_best_next_response

  i = 0

  for sentence in conversation:
    if i == 1:
      print(f'{characters[1]} : ' + sentence[buffer:])
      print()
    elif i % 2 == 0:
      print(f'{characters[0]} : ' + sentence)
      print()
    else:
      print(f'{characters[1]} : ' + sentence)
      print() 
    i += 1


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test'], ['This', 'is', 'train']]
candidate = ['this', 'is', 'a', 'train']
score = sentence_bleu(reference, candidate)
print(score)

0.7598356856515925


In [ ]:
movie_df_test = pd.read_csv(os.path.join(directory_path,'formatted_movie_filtered_lines_train.txt'), delimiter = '\n', error_bad_lines=False)
movie_lines_df_test = pd.DataFrame()


In [ ]:
movie_lines_df_test['CONVO'] = movie_df_test.iloc[:,0]
for i in range(len(movie_lines_df_test.iloc[:,0])):
  movie_lines_df_test.iloc[i,0] = movie_lines_df_test.iloc[i,0].replace('\t', ' ')
print(movie_lines_df_test.head(30))

                                                CONVO
0                                       no . stolen ?
1                              stolen ? not exactly .
2             you re not from l .a . huh ? new york .
3              new york . what motel you staying at ?
4            what motel you staying at ? u ho u tel .
5                 u ho u tel . okay what u ho u tel ?
6          you like being a hooker vivian ? it pays .
7                          it pays . so do day jobs .
8                 for the whole night ? for an hour .
9                       for an hour . you re joking .
10         you re joking . i never joke about money .
11                             three hundred . fine .
12                    fine . what s your name lover .
13  what s your name lover . edward . my name is e...
14  edward . my name is edward . edward . . . you ...
15                        get in . this side . what ?
16             what ? you know how to drive a stick ?
17                      off 

In [ ]:
movie_lines_df_test['CONVO'] = movie_df_test.iloc[:,0]

for i in range(len(movie_lines_df_test.iloc[:,0])):
  movie_lines_df_test.iloc[i,0] = movie_lines_df_test.iloc[i,0].replace('\t', ' ')

movie_lines_df_test['CONVO'] = movie_lines_df_test.CONVO.str.lower()

In [ ]:
test_references = []
for i in range(len(movie_lines_df_test.iloc[:])):
  test_references.append(movie_lines_df_test.iloc[i][0])

print(test_references)

['no . stolen ?', 'stolen ? not exactly .', 'you re not from l .a . huh ? new york .', 'new york . what motel you staying at ?', 'what motel you staying at ? u ho u tel .', 'u ho u tel . okay what u ho u tel ?', 'you like being a hooker vivian ? it pays .', 'it pays . so do day jobs .', 'for the whole night ? for an hour .', 'for an hour . you re joking .', 'you re joking . i never joke about money .', 'three hundred . fine .', 'fine . what s your name lover .', 'what s your name lover . edward . my name is edward .', 'edward . my name is edward . edward . . . you got it !', 'get in . this side . what ?', 'what ? you know how to drive a stick ?', 'off the boulevard . exactly .', 'room service . what do you drink ? diet coke .', 'diet coke . send up a bottle of crystal champagne .', 'send up a bottle of crystal champagne . can i have some chips ?', 'do you plan everything ? always .', 'let s talk some more first . right . so ! in town on business ?', 'right . so ! in town on business ? 

In [ ]:
from tqdm import tqdm
bleu_scores = []
for i in tqdm(range(1000,len(movie_lines_df_test.iloc[:])-9500)):
  output = predict(movie_lines_df_test.iloc[i][0], 'left to right')
  score = sentence_bleu(test_references, word_tokenize(output))
  #print(output, ' : ', score)
  bleu_scores.append(score)


100%|██████████| 132/132 [05:42<00:00,  2.60s/it]


In [ ]:
import numpy as np
bleu_scores_array = np.array(bleu_scores)
print(bleu_scores_array.mean())